In [32]:
import torch
import torch.nn as nn
import numpy as np



import numpy as np
import copy
import matplotlib.pyplot as plt


class boxFilters(nn.Module):
    def __init__(self):
        super(boxFilters, self).__init__()
        self.alpha = nn.Parameter(torch.tensor(1.0))
        self.a = nn.Parameter(torch.tensor(2.0))
        self.b = nn.Parameter(torch.tensor(3.0))
        self.c = nn.Parameter(torch.tensor(2.0))
        self.d = nn.Parameter(torch.tensor(6.0))
    
    def setAlpha(self,alpha):
        self.alpha = alpha
    
    def setABCD(self,a,b,c,d):
        self.a = a
        self.b = b
        self.c = c
        self.d = d
    
    def printParams(self):
        print("A:",self.a,"B:",self.b,"C:",self.c,"D:",self.d, "Alpha:", self.alpha)








In [108]:


# Convolution using box filters (1 Integral Image, n box filters)
# Need to write backprop for the forward pass 
class BoxFilterConvolution(nn.Module):
    def __init__(self, box_filters):
        super(BoxFilterConvolution, self).__init__()
        self.box_filters = box_filters

    def forward(self, integral_image):
        output = torch.zeros_like(integral_image)
        H, W = integral_image.shape[-2:]
        for box_filter in self.box_filters:
            a, b, c, d = int(box_filter.a), int(box_filter.b), int(box_filter.c), int(box_filter.d)
            if b >= a and d >= c:  # Check if the box filter is valid
                # Compute the sum within the box filter for each position in the image
                for i in range(H):
                    for j in range(W):
                        output[..., i, j] += integral_image[..., max(0, i+d-1):min(H, i+b), max(0, j+c-1):min(W, j+a)].sum()
            else:
                print(f"Invalid box filter: a={a}, b={b}, c={c}, d={d}")
        return output


# CNN Network with box filters
class cnnBox(nn.Module):

    def __init__(self, nBoxes):
        super(cnnBox, self).__init__()
        self.boxes = 10
        
        self.boxes = nn.ModuleList([boxFilters() for _ in range(self.boxes)])

        self.conv = BoxFilterConvolution(self.boxes)
        self.fc = nn.Linear(1024,128)
            

    def forward(self,x):
        x = x.cumsum(dim=-2).cumsum(dim=-1)
        x = self.conv(x)
        x = nn.Flatten()(x)
        x = self.fc(x)
        return x
        
        



In [109]:
convLayer = cnnBox(10)


In [110]:
convLayer.boxes[0].a

Parameter containing:
tensor(2., requires_grad=True)

In [120]:
for i,j in convLayer.named_parameters():
    print(i,j)

boxes.0.alpha Parameter containing:
tensor(1., requires_grad=True)
boxes.0.a Parameter containing:
tensor(2., requires_grad=True)
boxes.0.b Parameter containing:
tensor(3., requires_grad=True)
boxes.0.c Parameter containing:
tensor(2., requires_grad=True)
boxes.0.d Parameter containing:
tensor(6., requires_grad=True)
boxes.1.alpha Parameter containing:
tensor(1., requires_grad=True)
boxes.1.a Parameter containing:
tensor(2., requires_grad=True)
boxes.1.b Parameter containing:
tensor(3., requires_grad=True)
boxes.1.c Parameter containing:
tensor(2., requires_grad=True)
boxes.1.d Parameter containing:
tensor(6., requires_grad=True)
boxes.2.alpha Parameter containing:
tensor(1., requires_grad=True)
boxes.2.a Parameter containing:
tensor(2., requires_grad=True)
boxes.2.b Parameter containing:
tensor(3., requires_grad=True)
boxes.2.c Parameter containing:
tensor(2., requires_grad=True)
boxes.2.d Parameter containing:
tensor(6., requires_grad=True)
boxes.3.alpha Parameter containing:
tensor(

In [116]:
a = convLayer((torch.ones((1,1,32,32))))

print(a)

label = torch.ones((1,128))

lossFunction = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convLayer.parameters(),lr = 0.01)

optimizer.zero_grad

loss = lossFunction(a,label)

loss.backward()

optimizer.step()




tensor([[-6.7407e-03,  3.8544e-05, -1.7569e-02,  6.6483e-04, -4.5048e-03,
         -1.0810e-02,  1.0309e-02, -5.1756e-04, -9.6741e-03,  4.6584e-03,
         -1.7217e-02, -1.7985e-02,  1.5219e-02,  1.3098e-02, -3.5450e-03,
         -1.2223e-02,  2.3422e-03,  8.6034e-03, -1.1268e-02,  3.6759e-03,
         -1.5728e-02, -1.5234e-02,  1.8618e-02,  4.2981e-03, -4.7841e-03,
          6.0257e-03, -3.8544e-03, -1.1747e-02, -5.9944e-04,  1.5845e-03,
         -5.8710e-03, -1.5349e-02, -9.0407e-04,  1.7184e-02,  6.3869e-03,
         -1.5729e-02,  1.4257e-02, -1.3998e-03,  1.2017e-02,  8.4893e-03,
          1.6917e-02,  8.4618e-03, -1.0028e-02,  1.5290e-03, -4.7018e-03,
         -1.0959e-02, -1.4157e-02,  3.5690e-03, -2.0050e-03,  9.7287e-03,
          4.1273e-03,  5.8265e-05, -8.9195e-04,  1.5278e-02, -1.6389e-02,
         -5.2448e-03, -1.0019e-02, -1.2031e-02, -9.1731e-03,  1.8046e-02,
          1.5077e-02,  9.4165e-03,  4.3142e-03, -1.0608e-02,  1.3899e-02,
         -1.4344e-02, -1.6315e-02, -5.

In [118]:
convLayer.boxes[0].a

Parameter containing:
tensor(2., requires_grad=True)